In [19]:
# check if display name validation works

import os
import sys

os.environ["LCD_COLUMNS"] = "16"

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

